<a href="https://colab.research.google.com/github/Azenros/side-colabs/blob/main/hbcb2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Auto-generating a leaderboard (and other tools) for HBCB 2021 (hbcb.caard.co)
#   last updated: May 30, 2021
#   original at https://colab.research.google.com/drive/1CGGlooH7wo3OYvfA6RmrP7nZa7vIECfm?usp=sharing

# Requirements:
#    - All scores are in individual tabs in a SINGLE spreadsheet 
#    - ^ said spreadsheet must only contain scoresheets
#    - Names and scores must be a single cell each, in a constant location 
#      (same throughout all sheets, intended for copied judge sheets)
#    - If you have a leaderboard sheet, make sure that the leaderboard sheet
#      is in a SEPERATE spreadsheet with one tab (if you don't have a 
#      leaderboard sheet, this will make one for you as a new tab in the same
#      sheet)
#         - 3 columns required for team name, score, total ranking
#         - +4 columns if you include rankings for vox, art, mix, anim 

# Current Bugs/Errors:
#    - Bug: bad user input will cause data to overwrite existing data in cells

In [3]:
## Authorization (run this block first!)
!pip install --upgrade gspread
import gspread
from google.colab import auth
from google.colab import drive
from google.colab import output
from oauth2client.client import GoogleCredentials

# authorization must be done before you can run anything below
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())
output.clear()

print('Verification completed')


Verification completed


In [16]:
## Variables

# id of spreadsheet to pull data from (long string of alphanumerals in the link)
spreadsheet_key = '1KX5U8PqH95HZ-eJeG7dFz6dqr7kSCqWibRszYUbkTR0'
# if referencing a merged cell, use the top-left corner
teamName_location = 'B3' 
teamScore_location = 'M33'

# if you already have a leaderboard sheet prepped, set this to "True"
hasLeaderBoardSheet = False
# if hasLeaderBoardSheet is true, use these variables to place the data
leaderboard_key = ''
lb_topLeftCorner_location = ''

# do you include placement rankings for individual categories?
# (vox, mix, art, anim); if yes, set this to "True"
ranks = True
# locations of total category scores
vox_score_loc = 'G11'
mix_score_loc = 'O11'
art_score_loc = 'G18'
vid_score_loc = 'N18'

print('Variables Set')
# after running this block run the block below

Variables Set


In [17]:
## Sub-function definitions
def setRanks(name_list, check_ind, set_ind):
  temp_rank = 0
  temp_score = 0
  tie_count=0
  for team in name_list:
    if (temp_score != team[check_ind]):
      temp_rank += tie_count + 1
      tie_count = 0
    else:
      tie_count += 1
    temp_score = team[check_ind]
    team[set_ind] = temp_rank

def fillList(sheet_list, return_list, var_list):
  for sheet in sheet_list:
    temp = []
    for var in var_list:
      temp.append(sheet.acell(var).value)
    return_list.append(temp)

# set ranks to a certain list.
def addRanks(name_list, score_loc):
  ind_rank = []
  fillList(sheet_list, ind_rank, score_loc)
  ind = 1;
  while (ind < 5):
    ind_rank.sort(key = lambda x: float(x[ind]), reverse=True)
    setRanks(ind_rank, ind, ind)
    ind += 1
  ind_dict = {}
  for team in ind_rank:
    ind_dict[team[0]] = team[1:]
  for team in name_list:
    team.extend(ind_dict[team[0]])

print("Functions defined")
# after running this block run the block below

Functions defined


In [ ]:
## Leaderboard Function

# run the above two blocks of code before running this one

# this block of code will throw an error if a leaderboard tab is already
# made on the scoresheet

# open the sheet and get list of sheets
ws = gc.open_by_key(spreadsheet_name)
sheet_list = ws.worksheets()
name_list = []

# gets data from each team
name_vars = [teamName_location, teamScore_location]
fillList(sheet_list, name_list, name_vars)
for name in name_list:
  name.append(0)

print("Name list created")

# sort from highest to lowest total score
name_list.sort(key = lambda x: float(x[1]), reverse=True)
team_count = len(name_list)

# getting overall ranks
setRanks(name_list, 1, 2)
print("Overall ranks done")

# getting category ranks
if (ranks):
  score_loc = [teamName_location, vox_score_loc, art_score_loc, 
                                  mix_score_loc, vid_score_loc]
  addRanks(name_list, score_loc)
  print("Individual ranks done")


# inserts values into a new/existing sheet
extend = 22
titles = ['Team Name', 'Score', 'Rank (Overall)']
if (ranks):
  titles += ['Vox Rank', 'Art Rank','Mix Rank', 'Vid Rank']

if (hasLeaderBoardSheet == False):
  name_list.insert(0,titles)
  range_size = 4 + len(name_list) + extend
  col_begin = 'B'
  range = col_begin + '2:' + chr(ord(col_begin) + 
          len(name_list[0])) + str(3 + team_count)
  print("Range defind as " + range)
  col_size = 6 + extend
  if (ranks == True):
    col_size += 4

  le_ws = ws.add_worksheet(title='Leaderboard', 
                   rows=str(range_size),
                   cols=str(col_size))
  
  le_ws.update(range, name_list)
else:
  lb = gc.open_by_key(leaderboard_key).sheet1
  ch = lb_topLeftCorner_location
  range = ch + ':' + chr(ord(ch[0]) + 1) + str(int(ch[1]) + len(name_list))
  print(range)
  lb.update(range, name_list)

print("Leaderboard created successfully")

Name list created
Overall ranks done
Individual ranks done
B2:I20
Leaderboard created successfully


In [ ]:
# Spreadsheet Splitter
# 1 spreadsheet with N tabs -> N spreadsheets with 1 tab

spreadsheet_id = '1KX5U8PqH95HZ-eJeG7dFz6dqr7kSCqWibRszYUbkTR0'
folder_id = '1UR1ThfzQ8REgKPe0EQfuHOuucemvnlIs'

ws = gc.open_by_key(spreadsheet_id)

round_str = ": HBCB-R1 Results"

# for each sheet make new spreadsheet in folder and clone tab to it
for sheet in ws.worksheets():
  new_ss = gc.create(sheet.title + round_str, folder_id)
  new_ssId = new_ss.id
  sheet.copy_to(new_ssId)
  new_ss.del_worksheet(new_ss.sheet1)
  new_ss.sheet1.delete_row(5)
  new_ss.sheet1.update_title(sheet.title)
  print(sheet.title + ' copied into folder')

print("Complete!")

Team_Name copied into folder
Template copied into folder
test1 copied into folder
test2 copied into folder
test3 copied into folder
Leaderboard_test copied into folder
Complete!


In [15]:
## Judge Sheet Cloner using Team Data
# clones template sheet, then fills with each team's data
team_data_key = '1Amrey1K1ZRv4Z3YD07hWB5PhUavbN7lJZdnz8M6szSg'
clone_sheet_key = '1KX5U8PqH95HZ-eJeG7dFz6dqr7kSCqWibRszYUbkTR0'

hyperl = '=HYPERLINK(\"'

td = gc.open_by_key(team_data_key)
cs = gc.open_by_key(clone_sheet_key)

numgroups = 2
for index in range(2, numgroups + 2):
  # locations to grab info from team data sheet
  team_name = td.sheet1.acell("C" + str(index)).value
  links = [hyperl + td.sheet1.acell("D" + str(index)).value + '", "Video Link")',
           hyperl + td.sheet1.acell("E" + str(index)).value + '", "Alternate Link")', 
           hyperl + td.sheet1.acell("F" + str(index)).value + '", "Script & Theme")', 
           hyperl + td.sheet1.acell("G" + str(index)).value + '", "Art Folder")',
           hyperl + td.sheet1.acell("H" + str(index)).value + '", "Audio Folder")',
           hyperl + td.sheet1.acell("I" + str(index)).value + '", "Asset Folder")']

  # makes new sheet and fills in hyperlinks
  cs.sheet1.copy_to(clone_sheet_key)
  ns = cs.get_worksheet(index - 1)
  ns.update_title(team_name)
  ns.update_cell(3, 2, team_name)
  l = 0
  for col in range(3,15,2):
    ns.update_cell(5,col, links[l])
    l += 1

  print("Spreadsheet created for " + team_name)
